In [5]:
import csv
import cv2
import numpy as np
%matplotlib inline

lines = []
with open ('./mydata/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
    
images = []
measurements = []

for line in lines:
    source_path = line[0]
    if source_path == 'center':
        continue
    filename = source_path.split('/')[-1]
    current_path = './mydata/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)

X_train = np.array(images)
y_train = np.array(measurements)

In [6]:
from keras.models import Sequential
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.optimizers import Adam
from keras.layers.advanced_activations import ELU

model = Sequential()

# Normalize
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160, 320, 3)))

# Add 5 convolutional layers
model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2,2)))
model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2,2)))
model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2,2)))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Conv2D(64, 3, 3, activation='elu'))

# Add a flatten layer
model.add(Flatten())

# Add 3 fully connected layers
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))

# Add a fully connected output layers
model.add(Dense(1))

# Compile and train the model
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=5)

# Print model summary
print(model.summary)

# Save model
model.save('model.h5')

Train on 1528 samples, validate on 382 samples
Epoch 1/5
1528/1528 [==============================] - 7s - loss: 0.0951 - val_loss: 0.0089
Epoch 2/5
1528/1528 [==============================] - 6s - loss: 0.0064 - val_loss: 0.0044
Epoch 3/5
1528/1528 [==============================] - 6s - loss: 0.0039 - val_loss: 0.0044
Epoch 4/5
1528/1528 [==============================] - 6s - loss: 0.0024 - val_loss: 0.0043
Epoch 5/5
1528/1528 [==============================] - 6s - loss: 0.0015 - val_loss: 0.0051
<bound method Container.summary of <keras.models.Sequential object at 0x7f7ab3b78390>>
